[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jvh95xBAniCw_CENrfCFsAtZAIvAMDt4?usp=sharing)

In [ ]:
!pip install pytorch-pretrained-bert
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from scipy.spatial.distance import cosine
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

     |████████████████████████████████| 133kB 5.0MB/s 
     |████████████████████████████████| 645kB 16.2MB/s 


The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 213450/213450 [00:00<00:00, 1131197.97B/s]


In [ ]:
def compare(food):
  # Define a new example sentence with multiple meanings of the word "bank"
  text = food[0]+" "+food[1]+" "+food[2]

  # Add the special tokens.
  marked_text = "[CLS] " + text + " [SEP]"

  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Mark each of the 22 tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Load pre-trained model (weights)
  model = BertModel.from_pretrained('bert-base-cased')

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()
  # Predict hidden states features for each layer
  with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
  # Concatenate the tensors for all layers. We use `stack` here to
  # create a new dimension in the tensor.
  token_embeddings = torch.stack(encoded_layers, dim=0)

  token_embeddings.size()
  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)

  token_embeddings.size()
  # Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)

  token_embeddings.size()
  # Stores the token vectors, with shape [22 x 3,072]
  token_vecs_cat = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

   
  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

 
  for i, token_str in enumerate(tokenized_text):
    print (i, token_str)
  # Calculate the cosine similarity between the word bank 
  # in "bank robber" vs "river bank" (different meanings).
  diff_bank = 1 - cosine(token_vecs_sum[2], token_vecs_sum[1])

  # Calculate the cosine similarity between the word bank
  # in "bank robber" vs "bank vault" (same meaning).
  same_bank = 1 - cosine(token_vecs_sum[2], token_vecs_sum[3])

  print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
  print('Vector similarity for *different* meanings:  %.2f' % diff_bank)






In [ ]:
fruits = ["big", "large", "small"]

compare(fruits)



0 [CLS]
1 big
2 large
3 small
4 [SEP]
Vector similarity for  *similar*  meanings:  0.92
Vector similarity for *different* meanings:  0.89
